In [ ]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from moment.common import PATHS
from moment.utils.ucr_anomaly_archive_subset import ucr_anomaly_archive_subset

In [ ]:
EXPERIMENT_NAME = "supervised_anomaly_detection"

def get_model_results(model_name):
    if model_name == 'MOMENT_LP':
        # results_path = os.path.join(
        #     PATHS.RESULTS_DIR, EXPERIMENT_NAME, model_name, 'linear-probing')
        results_path = os.path.join(
            '/home/scratch/mgoswami/moment_results/', EXPERIMENT_NAME, 'MOMENT', 'linear-probing')
    if model_name == 'MOMENT_0':
        # results_path = os.path.join(
        #     PATHS.RESULTS_DIR, EXPERIMENT_NAME, model_name, 'linear-probing')
        results_path = os.path.join(
            '/home/scratch/mgoswami/moment_results/', "zero_shot_anomaly_detection")
    elif model_name in ['DGHL', 'AnomalyTransformer', 'TimesNet', 'GPT4TS', 'AnomalyNearestNeighbors']:
        # results_path = os.path.join(
        #     PATHS.RESULTS_DIR, EXPERIMENT_NAME, model_name, 'end-to-end')
        results_path = os.path.join(
            "/home/scratch/mgoswami/moment_results/", EXPERIMENT_NAME, model_name, 'end-to-end')

    datasets_without_results = []
    results = []
    for dataset_name in tqdm(ucr_anomaly_archive_subset, total=len(ucr_anomaly_archive_subset)): 
        metadata = dataset_name.split('/')[-1].split('_')
        data_id, data_name = metadata[0], metadata[3]

        if not os.path.exists(os.path.join(results_path, f"results_{data_id}_{data_name}.csv")):
            datasets_without_results.append(dataset_name)
            print(f"Missing results for {os.path.join(results_path, f'results_{data_id}_{data_name}.csv')}")
            continue
        
        best_result_path = os.path.join(results_path, f"results_{data_id}_{data_name}.csv")
        df = pd.read_csv(best_result_path, index_col=0).T

        if 'Model name' in df.columns:
            df['Run name'] = df['Model name']
        else: 
            df['Run name'] = f'MOMENT_0'
            df['ID'] = 0
        df['Model name'] = model_name
        df['Dataset name'] = data_name
        df['Dataset ID'] = data_id
        last_modified_time = int(os.path.getmtime(best_result_path))
        last_modified_time = datetime.utcfromtimestamp(last_modified_time).strftime('%d-%m-%Y %H:%M')
        df['Timestamp'] = last_modified_time
        
        results.append(df)

    results = pd.concat(results, axis=0).reset_index(drop=True)
    results = results[[
        'Model name', 'Dataset ID', 'Dataset name', 
        'Adj. Best F1', 'rAUCROC', 'rAUCPR', 'VUSROC', 
        'VUSPR', 'Run name', 'ID', 'Timestamp']]
    
    return results, datasets_without_results

In [ ]:
anomaly_transformer_results, datasets_without_results_at = get_model_results(model_name='AnomalyTransformer')
moment_0_results, datasets_without_results_moment = get_model_results(model_name='MOMENT_0')
moment_LP_results, datasets_without_results_moment = get_model_results(model_name='MOMENT_LP')
dghl_results, datasets_without_results_dghl = get_model_results(model_name='DGHL')
timesnet_results, datasets_without_results_timesnet = get_model_results(model_name='TimesNet')
gpt4ts_results, datasets_without_results_gpt4ts = get_model_results(model_name='GPT4TS')
ann_results, datasets_without_results_ann = get_model_results(model_name='AnomalyNearestNeighbors')

results = pd.concat([moment_0_results, moment_LP_results, gpt4ts_results, timesnet_results, anomaly_transformer_results, dghl_results, ann_results], axis=0)
results.to_csv("../../assets/results/finetuning/anomaly_detection.csv", index=False)

print(f"\n===Results shape===")
print(f"MOMENT (zero-shot): {moment_0_results.shape}")
print(f"MOMENT (linear probing): {moment_LP_results.shape}")
print(f"AnomalyTransformer: {anomaly_transformer_results.shape}")
print(f"DGHL: {dghl_results.shape}")
print(f"TimesNet: {timesnet_results.shape}")
print(f"GPT4TS: {gpt4ts_results.shape}")
print(f"AnomalyNearestNeighbors: {ann_results.shape}")
print(f"Total: {results.shape}")

In [ ]:
results.dropna(inplace=True)
print(f"Results: {results.shape}" )
results.head()

In [ ]:
r = results[['Model name', 'Dataset name', 'Adj. Best F1', 'VUSROC']]
r[['Adj. Best F1']] = r[['Adj. Best F1']].apply(pd.to_numeric, errors='coerce')
table = r.pivot_table(index='Dataset name', columns='Model name', values=['Adj. Best F1'], aggfunc='mean')
average_rank = table.rank(axis=1, method='average', ascending=False)

average_rank.describe()

In [ ]:
r = results[['Model name', 'Dataset name', 'Adj. Best F1', 'VUSROC']]
r[['VUSROC']] = r[['VUSROC']].apply(pd.to_numeric, errors='coerce')
table = r.pivot_table(index='Dataset name', columns='Model name', values=['VUSROC'], aggfunc='mean')
average_rank = table.rank(axis=1, method='average', ascending=False)

average_rank.describe()

### Results for the paper

In [ ]:
r = results[['Model name', 'Dataset name', 'Adj. Best F1', 'VUSROC']]
r[['Adj. Best F1', 'VUSROC']] = r[['Adj. Best F1', 'VUSROC']].apply(pd.to_numeric, errors='coerce')
table = r.pivot_table(index='Dataset name', columns='Model name', values=['Adj. Best F1', 'VUSROC'], aggfunc='mean')
table = table.round(2)
table

In [ ]:
table.to_latex("../../assets/results/zero_shot/anomaly_detection.tex", 
               multicolumn=True, multirow=True, multicolumn_format='c', float_format="%.3f")

In [ ]:
model_name = 'AnomalyNearestNeighbors' # MOMENT, AnomalyTransformer, DGHL, TimesNet, GPT4TS, AnomalyNearestNeighbors
metrics = ['Adj. Best F1', 'rAUCROC', 'rAUCPR', 'VUSROC', 'VUSPR']
results[results['Model name'] == model_name][metrics].astype(float).describe()

In [ ]:
model_name = 'MOMENT' # MOMENT, AnomalyTransformer, DGHL
results[results['Model name'] == model_name][metrics].astype(float).describe()

In [ ]:
model_name = 'DGHL' # MOMENT, AnomalyTransformer, DGHL
results[results['Model name'] == model_name][metrics].astype(float).describe()

In [ ]:
def box_plot(metric_name: str='Adj. Best F1'):
    data = [
    results[results['Model name'] == 'MOMENT'][metric_name].astype(np.float32),
    results[results['Model name'] == 'AnomalyTransformer'][metric_name].astype(np.float32),
    results[results['Model name'] == 'DGHL'][metric_name].astype(np.float32)]
    
    labels = ['MOMENT', 'Anomaly Transformer', 'DGHL']

    boxprops = dict(linestyle='-', linewidth=1, color='k')
    flierprops = dict(marker='o', markersize=12, markeredgecolor='darkgreen')
    medianprops = dict(linestyle='-', linewidth=2, color='blue')
    meanpointprops = dict(marker='D', markeredgecolor='black',
                        markerfacecolor='firebrick')
    meanlineprops = dict(linestyle='--', linewidth=2, color='red')

    _ = plt.boxplot(
        x=data, labels=labels, showmeans=True, 
        meanline=True, bootstrap=10000, notch=True, 
        boxprops=boxprops, flierprops=flierprops, 
        medianprops=medianprops, 
        meanprops=meanlineprops)
    
    if metric_name == 'Adj. Best F1':
        plt.ylabel("Adjusted Best $F_1$", fontsize=16)
    else:
        plt.ylabel(metric_name, fontsize=16)
    
    plt.xlabel("Models", fontsize=16)
    plt.ylim(0.0, None)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid(color='lightgray', linestyle='--', linewidth=0.5) 
    plt.savefig(f"../../assets/figures/finetuning/anomaly_detection_{metric_name}.png", 
                dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
box_plot(metric_name='Adj. Best F1')

In [ ]:
box_plot(metric_name='VUSROC')

In [ ]:
datasets_without_results_at

In [ ]:
datasets_without_results_dghl

In [ ]:
datasets_without_results_moment

### Experiment scripts

In [ ]:
script = ""
config = "../../configs/anomaly_detection/linear_probing.yaml"
gpu_id = 0
finetuning_mode = 'linear-probing'

for dataset_name in ucr_anomaly_archive_subset:
    script += f"python anomaly_detection.py --config {config} --gpu_id {gpu_id} --finetuning_mode {finetuning_mode} --dataset_names {dataset_name}\n"

print(script)